# Available data and models

> Abstract: Different Swarm datasets are available through each "collection" on the VirES server. Choosing a collection determines the subset of "measurement"-type products available, while "auxiliary"-type are always available. Geomagnetic "model"-type are available in connection with the MAG collections. These can be seen at https://viresclient.readthedocs.io/en/latest/available_parameters.html

In [ ]:
%load_ext watermark
%watermark -i -v -p viresclient,pandas,xarray,matplotlib

In [ ]:
from viresclient import SwarmRequest

## Querying available variables

viresclient has some tools to help you find out what data and models are available. First instantiate a request object, then use it to call methods:
```
request.available_collections()
request.available_measurements()
request.available_auxiliaries()
request.available_models()
```

In [ ]:
request = SwarmRequest()

### Data are organised into "collections"

To see a list of them with references:

In [ ]:
request.available_collections()

Just the names of the collections:

In [ ]:
request.available_collections(details=False)

The collections are grouped according to those containing identical variable names, e.g.  
`'MAG': ['SW_OPER_MAGA_LR_1B', 'SW_OPER_MAGB_LR_1B', 'SW_OPER_MAGC_LR_1B']`  
indicates that these three are all of "MAG" type, while the actual collection names such as `SW_OPER_MAGA_LR_1B` point to the specific dataset - in this case, from Swarm Alpha (A), while the others are from Bravo (B) and Charlie (C).

### Available "measurements"

To query the variable names possible from a given collection type:

In [ ]:
request.available_measurements("MAG")

In [ ]:
request.available_measurements("EFI")

In [ ]:
request.available_measurements("IBI")

In [ ]:
request.available_measurements("TEC")

In [ ]:
request.available_measurements("FAC")

In [ ]:
request.available_measurements("EEF")

In [ ]:
request.available_measurements("IPD")

### Available "auxiliaries"

These can be requested together with any collection

In [ ]:
request.available_auxiliaries()

### Available "models"

(and custom ones can be supplied as .shc files)

In [ ]:
request.available_models(details=False)

The Swarm models are named with a prefix indicating the source field (e.g. MCO for "Model of the Core field) and suffix indicating the processing chain (e.g. 2D for "Level 2 product, Comprehensive Inversion chain").

C: Comprehensive; D: Dedicated; F: Fast-track

|                     	| C 	| D 	| F 	|
|---------------------	|---	|---	|---	|
| MCO - Core          	| x 	| x 	| - 	|
| MMA - Magnetosphere 	| x 	| - 	| x 	|
| MIO - Ionosphere    	| x 	| x 	| - 	|
| MLI - Lithosphere   	| x 	| x 	| - 	|

MIO and MMA are divided into Primary and Secondary parts - primary for the external (to the Earth) ionospheric / magnetospheric field source, and secondary for the internally induced part in the upper mantle.

#### Check models of a certain type

e.g. the CI models:

In [ ]:
request.available_models("C", details=False)

... or MCO (core) models:

In [ ]:
request.available_models("MCO", details=True)

## Manipulation of models

Models can be manipulated: combining different models, limiting spherical harmonic (SH) series summation to a smaller range of SH degree. These composed models can be provided within the `models` kwarg in `request.set_products()`, e.g.

```python
from viresclient import SwarmRequest
request = SwarmRequest()
request.set_collection("SW_OPER_MAGA_LR_1B")
request.set_products(
    measurements=["F"],
    models=["MCO_MMA = 'MCO_SHA_2C' + 'MMA_SHA_2C-Primary' + 'MMA_SHA_2C-Secondary'"],
)
```

Model details can be found with `request.get_model_info(models=...)` which handles the same `models` input as `set_products()`. This information is returned as a dictionary.

1. The models should be provided as a list of strings, where each string defines a particular model.

In [ ]:
request.get_model_info(
    models=["MCO_SHA_2D", "MCO_SHA_2C"]
)

2.. Models can be combined to form a new model like:
```
"New_model = 'Model_1' + 'Model_2'"
```

In [ ]:
request.get_model_info(
    models=["MCO_MMA = 'MCO_SHA_2C' + 'MMA_SHA_2C-Primary' + 'MMA_SHA_2C-Secondary'"]
)

3. Limiting the SH degree range can be done with:
```
Model_name(min_degree=x, max_degree=y)
```

4. Your own .shc format model can be provided as a file, the model for which is then accessible within VirES under the name `"Custom_Model"`. NB: in this case you will also need to provide the `custom_model` kwarg in `request.set_products()` - the model is not persistently stored on the server.

5. New models can be defined successively in the list, using names that have been defined earlier in the list.

In [ ]:
# Fetch an example file to use
url = "http://www.spacecenter.dk/files/magnetic-models/LCS-1/LCS-1.shc"
file_name = "LCS-1.shc"
import urllib.request
urllib.request.urlretrieve(url, file_name);

In [ ]:
# Demonstrates:
#  limiting SH degree
#  providing your own model file
#  referring to defined models
request.get_model_info(
    models=[
        "MLI_SHA_2D = MLI_SHA_2D(min_degree=1, max_degree=80)",
        "LCS = Custom_Model(min_degree=1, max_degree=80)",
        "LCS-SwarmMLI = LCS - MLI_SHA_2D"
    ],
    custom_model=file_name,
)

It is also possible to provide the models as a dictionary instead of a list.

In [ ]:
request.get_model_info(
    models={
        "MLI_SHA_2D": "MLI_SHA_2D(min_degree=1, max_degree=80)",
        "LCS": "Custom_Model(min_degree=1, max_degree=80)",
        "LCS-SwarmMLI": "LCS - MLI_SHA_2D"
    },
    custom_model="LCS-1.shc"
)